<a href="https://colab.research.google.com/github/alex-sal1991/footy/blob/main/Premier_League_Data_(Updated).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import json
import requests
import pandas as pd
import time

Mounted at /content/drive


In [2]:
API_KEY = "xxxxxxxxxxxxxx"

'''
This function makes an API call to gather data on previous seasons for the premier league. The for loop will loop through the json file and populate the list with the season_info dictionaries.
You can then select the season_id associated with the season you want to gather data for.
'''

def retrieve_season():
  url = "https://api.sportradar.com/soccer/trial/v4/en/competitions/sr%3Acompetition%3A17/seasons.json"

  headers = {
      "accept": "application/json",
      "x-api-key": API_KEY
  }

  response = requests.get(url, headers=headers).json()

  seasons = []

  for i in range(len(response['seasons'])):
    season_info = {
      'id': response['seasons'][i]['id'].split(':')[-1],
      'year': response['seasons'][i]['year'],
      'name': response['seasons'][i]['name']
    }

    seasons.append(season_info)


  return seasons

league_seasons = retrieve_season()
league_seasons

[{'id': '105353', 'year': '23/24', 'name': 'Premier League 23/24'},
 {'id': '118689', 'year': '24/25', 'name': 'Premier League 24/25'},
 {'id': '130281', 'year': '25/26', 'name': 'Premier League 25/26'}]

In [3]:
import time

'''
The season_schedule function takes in the season_id which can be retrieved from the retrieve_season function return and the API_KEY. This will return a json file that displays the season schedule for the premier league and results if the
matches have been played.
'''

def season_schedule(season_id, API_KEY):
  url = f"https://api.sportradar.com/soccer/trial/v4/en/seasons/sr%3Aseason%3A{season_id}/schedules.json"

  headers = {
    "accept": "application/json",
    "x-api-key": f"{API_KEY}"
}

  response = requests.get(url, headers=headers).json()
  time.sleep(5)

  return response

premier_league = [season_schedule(118689, API_KEY=API_KEY)]

In [ ]:
# I want to take a look at the structure of the json before extracting the data I need to create a dataframe displaying basic match statistics

premier_league[0]['schedules'][0]

{'sport_event': {'id': 'sr:sport_event:50849967',
  'start_time': '2024-08-16T19:00:00+00:00',
  'start_time_confirmed': True,
  'sport_event_context': {'sport': {'id': 'sr:sport:1', 'name': 'Soccer'},
   'category': {'id': 'sr:category:1',
    'name': 'England',
    'country_code': 'ENG'},
   'competition': {'id': 'sr:competition:17',
    'name': 'Premier League',
    'gender': 'men'},
   'season': {'id': 'sr:season:118689',
    'name': 'Premier League 24/25',
    'start_date': '2024-08-16',
    'end_date': '2025-05-25',
    'year': '24/25',
    'competition_id': 'sr:competition:17'},
   'stage': {'order': 1,
    'type': 'league',
    'phase': 'regular season',
    'start_date': '2024-08-16',
    'end_date': '2025-05-25',
    'year': '24/25'},
   'round': {'number': 1},
   'groups': [{'id': 'sr:league:84075', 'name': 'Premier League 24/25'}]},
  'coverage': {'type': 'sport_event',
   'sport_event_properties': {'lineups': True,
    'formations': False,
    'venue': True,
    'extended_

In [4]:
# the league variable drills down into each dictionary that contains information about each match. This variable will help create different tables such as venue and teams.

league = premier_league[0]['schedules']

In [5]:
'''
The team_info function takes in the league variable as a parameter loops through the dictionaries and populates the team_info dictionary which is appended to the teams
list. The purpose of this function is to create a list of basic information on teams that can be used to build a teams table.
'''

def team_info(league):
  teams = []

  for i in range(len(league)):
      team_info = {
                "team_id": league[i]["sport_event"]["competitors"][0]["id"].split(":")[-1],
                "competition_id": league[i]["sport_event"]["sport_event_context"]["competition"]["id"].split(":")[-1],
                "team_name": league[i]["sport_event"]["competitors"][0]["name"],
                "venue_id": league[i]["sport_event"]["venue"]["id"].split(":")[-1],
                "city": league[i]["sport_event"]["venue"]["city_name"]
            }

      if team_info not in teams:
          teams.append(team_info)
      else:
          continue

  return teams

team_data = team_info(league=league)

team_data

[{'team_id': '35',
  'competition_id': '17',
  'team_name': 'Manchester United',
  'venue_id': '9',
  'city': 'Manchester'},
 {'team_id': '32',
  'competition_id': '17',
  'team_name': 'Ipswich Town',
  'venue_id': '2188',
  'city': 'Ipswich'},
 {'team_id': '42',
  'competition_id': '17',
  'team_name': 'Arsenal FC',
  'venue_id': '624',
  'city': 'London'},
 {'team_id': '48',
  'competition_id': '17',
  'team_name': 'Everton FC',
  'venue_id': '12',
  'city': 'Liverpool'},
 {'team_id': '39',
  'competition_id': '17',
  'team_name': 'Newcastle United',
  'venue_id': '765',
  'city': 'Newcastle upon Tyne'},
 {'team_id': '14',
  'competition_id': '17',
  'team_name': 'Nottingham Forest',
  'venue_id': '2230',
  'city': 'Nottingham'},
 {'team_id': '37',
  'competition_id': '17',
  'team_name': 'West Ham United',
  'venue_id': '18379',
  'city': 'London'},
 {'team_id': '50',
  'competition_id': '17',
  'team_name': 'Brentford FC',
  'venue_id': '53349',
  'city': 'London'},
 {'team_id': '3

In [ ]:
premier_league_teams = pd.DataFrame(team_data)
premier_league_teams.to_csv('/content/drive/My Drive/prem_teams.csv', index=False)

premier_league_teams

Mounted at /content/drive


,team_id,competition_id,team_name,venue_id,city
0,35,17,Manchester United,9,Manchester
1,32,17,Ipswich Town,2188,Ipswich
2,42,17,Arsenal FC,624,London
3,48,17,Everton FC,12,Liverpool
4,39,17,Newcastle United,765,Newcastle upon Tyne
5,14,17,Nottingham Forest,2230,Nottingham
6,37,17,West Ham United,18379,London
7,50,17,Brentford FC,53349,London
8,38,17,Chelsea FC,799,London
9,31,17,Leicester City,2183,Leicester


In [ ]:
def venue_info(league):
    venues = []

    for i in range(len(league)):
        venue = {
            "venue_id": league[i]["sport_event"]["venue"]["id"].split(":")[-1],
            "team_id": league[i]["sport_event"]["competitors"][0]["id"].split(":")[-1],
            "name": league[i]["sport_event"]["venue"]["name"],
            "country": league[i]["sport_event"]["venue"]["country_name"],
            "city": league[i]["sport_event"]["venue"]["city_name"],
            "capacity": league[i]["sport_event"]["venue"]["capacity"]
        }

        if venue not in venues:
            venues.append(venue)
        else:
            continue

    return venues

venue_data = venue_info(league=league)

premier_league_venues = pd.DataFrame(venue_data)
premier_league_venues.to_csv('/content/drive/My Drive/prem_venues.csv', index=False)
premier_league_venues


,venue_id,team_id,name,country,city,capacity
0,9,35,Old Trafford,England,Manchester,75635
1,2188,32,Portman Road Stadium,England,Ipswich,30017
2,624,42,Emirates Stadium,England,London,60704
3,12,48,Goodison Park,England,Liverpool,39572
4,765,39,St. James' Park,England,Newcastle upon Tyne,52258
5,2230,14,City Ground,England,Nottingham,30404
6,18379,37,London Stadium,England,London,66000
7,53349,50,Brentford Community Stadium,England,London,17250
8,799,38,Stamford Bridge,England,London,40341
9,2183,31,King Power Stadium,England,Leicester,32261


In [ ]:
league[0]['sport_event_status']

{'status': 'closed',
 'match_status': 'ended',
 'home_score': 1,
 'away_score': 0,
 'winner_id': 'sr:competitor:35',
 'period_scores': [{'home_score': 0,
   'away_score': 0,
   'type': 'regular_period',
   'number': 1},
  {'home_score': 1, 'away_score': 0, 'type': 'regular_period', 'number': 2}]}

In [7]:
'''
This function accepts a list and loops through each dictionary extracting the values needed to populate the match_details dictionary
inside the function which is stored in the matches list and returned.
'''

def match_stats(league):
  matches = []

  for i in range(len(league)):
      if league[i]['sport_event_status']['status'] != "postponed":
          match_data = {
              "competition_id": league[i]['sport_event']['sport_event_context']["competition"]['id'].split(":")[-1],
              "competition_name": league[i]['sport_event']['sport_event_context']["competition"]['name'],
              "season_id": league[i]['sport_event']['sport_event_context']['season']['id'].split(":")[-1],
              "match_id": league[i]['sport_event']['id'].split(":")[-1],
              "kickoff": league[i]['sport_event']['start_time'],
              "venue": league[i]['sport_event']['venue']['id'].split(":")[-1],
              "city": league[i]['sport_event']['venue']['city_name'],
              "hometeam_id": league[i]['sport_event']['competitors'][0]['id'].split(":")[-1],
              "awayteam_id": league[i]['sport_event']['competitors'][1]['id'].split(":")[-1],
              "hometeam_halftime_score": league[i]['sport_event_status']['period_scores'][0]['home_score'],
              "awayteam_halftime_score": league[i]['sport_event_status']['period_scores'][1]['away_score'],
              "hometeam_fulltime_score": league[i]['sport_event_status']['home_score'],
              "awayteam_fulltime_score": league[i]['sport_event_status']['away_score'],
        }

          matches.append(match_data)
  return matches

fixtures = match_stats(league=league)
premier_league_fixtures = pd.DataFrame(fixtures)
# premier_league_fixtures.to_csv('/content/drive/My Drive/prem_fixtures.csv', index=False)

premier_league_fixtures.head(10)

,competition_id,competition_name,season_id,match_id,kickoff,venue,city,hometeam_id,awayteam_id,hometeam_halftime_score,awayteam_halftime_score,hometeam_fulltime_score,awayteam_fulltime_score
0,17,Premier League,118689,50849967,2024-08-16T19:00:00+00:00,9,Manchester,35,43,0,0,1,0
1,17,Premier League,118689,50849969,2024-08-17T11:30:00+00:00,2188,Ipswich,32,44,0,0,0,2
2,17,Premier League,118689,50849971,2024-08-17T14:00:00+00:00,624,London,42,3,1,0,2,0
3,17,Premier League,118689,50849973,2024-08-17T14:00:00+00:00,12,Liverpool,48,30,0,1,0,3
4,17,Premier League,118689,50849975,2024-08-17T14:00:00+00:00,765,Newcastle upon Tyne,39,45,1,0,1,0
5,17,Premier League,118689,50849977,2024-08-17T14:00:00+00:00,2230,Nottingham,14,60,1,0,1,1
6,17,Premier League,118689,50849979,2024-08-17T16:30:00+00:00,18379,London,37,40,1,1,1,2
7,17,Premier League,118689,50849981,2024-08-18T13:00:00+00:00,53349,London,50,7,1,0,2,1
8,17,Premier League,118689,50849983,2024-08-18T15:30:00+00:00,799,London,38,17,0,1,0,2
9,17,Premier League,118689,50849985,2024-08-19T19:00:00+00:00,2183,Leicester,31,33,0,1,1,1


In [9]:
'''
This function will extract the match_ids from the fixtures table added as a parameter.
'''

def extract_match_id(fixtures):
  match_ids = []

  for id in fixtures['match_id']:
    match_ids.append(id)

  return match_ids

premier_league_match_ids = extract_match_id(fixtures=premier_league_fixtures)
len(premier_league_match_ids)

380

In [10]:
'''
This function takes in a list of match_ids and loops through them adding each into the url. Each iteration (id) requests an API call to generate the more detailed match statistics.
Each dictionary will be stored in the detailed_match_stats dictionary for further extraction.
'''

def extract_detailed_match_statistics(match_ids):
  detailed_match_stats = []

  for id in match_ids:
      url = f"https://api.sportradar.com/soccer/trial/v4/en/sport_events/sr%3Asport_event%3A{id}/summary.json"

      headers = {
                  "accept": "application/json",
                  "x-api-key": f"{API_KEY}"
              }

      match_statistics = requests.get(url, headers=headers).json()
      detailed_match_stats.append(match_statistics)
      time.sleep(5)

  return detailed_match_stats


In [11]:
premier_league_detailed_match_stats = extract_detailed_match_statistics(match_ids=premier_league_match_ids)

In [12]:
# Checking that there are 380 dictionaries in the premier_league_detailed_match_stats list

len(premier_league_detailed_match_stats)

380

In [13]:
'''
This function takes in a list (premier_league_detailed_match_stats), performs a similar task to the match_stats function that loops through the dictionaries in the list
and populates the match_details dictionary before appending each dictionary to the list in function and returning the list.
'''

def extract_match_details(match_data):

  all_match_details = []

  for i in range(len(match_data)):

        match_details = {

                    "competition_id": match_data[i]["sport_event"]["sport_event_context"]["competition"]["id"].split(":")[-1],
                    "match_id": match_data[i]["sport_event"]["id"].split(":")[-1],
                    "matchday": match_data[i]['sport_event']['sport_event_context']['round']['number'],
                    "venue_id": match_data[i]["sport_event"]["venue"]["id"].split(":")[-1],
                    "referee": match_data[i]["sport_event"]["sport_event_conditions"]["referees"][0]["name"],
                    "hometeam_id": match_data[i]["sport_event"]["competitors"][0]["id"].split(":")[-1],
                    "awayteam_id": match_data[i]["sport_event"]["competitors"][1]["id"].split(":")[-1],
                    "hometeam_ball_possession": match_data[i]["statistics"]["totals"]["competitors"][0]["statistics"]["ball_possession"],
                    "awayteam_ball_possession": match_data[i]["statistics"]["totals"]["competitors"][1]["statistics"]["ball_possession"],
                    "hometeam_shots_on_target": match_data[i]["statistics"]["totals"]["competitors"][0]["statistics"]["shots_on_target"],
                    "awayteam_shots_on_target": match_data[i]["statistics"]["totals"]["competitors"][1]["statistics"]["shots_on_target"],
                    "hometeam_shots_off_target": match_data[i]["statistics"]["totals"]["competitors"][0]["statistics"]["shots_off_target"],
                    "awayteam_shots_off_target": match_data[i]["statistics"]["totals"]["competitors"][1]["statistics"]["shots_off_target"],
                    "hometeam_shots_blocked": match_data[i]["statistics"]["totals"]["competitors"][0]["statistics"]["shots_blocked"],
                    "awayteam_shots_blocked": match_data[i]["statistics"]["totals"]["competitors"][1]["statistics"]["shots_blocked"],
                    "hometeam_shots_saved": match_data[i]["statistics"]["totals"]["competitors"][0]["statistics"]["shots_saved"],
                    "awayteam_shots_saved": match_data[i]["statistics"]["totals"]["competitors"][1]["statistics"]["shots_saved"],
                    "hometeam_offsides": match_data[i]["statistics"]["totals"]["competitors"][0]["statistics"]["offsides"],
                    "awayteam_offsides": match_data[i]["statistics"]["totals"]["competitors"][1]["statistics"]["offsides"],
                    "hometeam_corner_kicks": match_data[i]["statistics"]["totals"]["competitors"][0]["statistics"]["corner_kicks"],
                    "awayteam_corner_kicks": match_data[i]["statistics"]["totals"]["competitors"][1]["statistics"]["corner_kicks"],
                    "hometeam_free_kicks": match_data[i]["statistics"]["totals"]["competitors"][0]["statistics"]["free_kicks"],
                    "awayteam_free_kicks": match_data[i]["statistics"]["totals"]["competitors"][1]["statistics"]["free_kicks"],
                    "hometeam_cards_given": match_data[i]["statistics"]["totals"]["competitors"][0]["statistics"]["cards_given"],
                    "awayteam_cards_given": match_data[i]["statistics"]["totals"]["competitors"][1]["statistics"]["cards_given"],
                    "hometeam_substitutions": match_data[i]["statistics"]["totals"]["competitors"][0]["statistics"]["substitutions"],
                    "awayteam_substitutions": match_data[i]["statistics"]["totals"]["competitors"][1]["statistics"]["substitutions"],
                    "hometeam_injuries": match_data[i]["statistics"]["totals"]["competitors"][0]["statistics"]["injuries"],
                    "awayteam_injuries": match_data[i]["statistics"]["totals"]["competitors"][1]["statistics"]["injuries"],
                    "hometeam_yellow_cards": match_data[i]["statistics"]["totals"]["competitors"][0]["statistics"]["yellow_cards"],
                    "awayteam_yellow_cards": match_data[i]["statistics"]["totals"]["competitors"][1]["statistics"]["yellow_cards"],
                    "hometeam_yellow_red_cards": match_data[i]["statistics"]["totals"]["competitors"][0]["statistics"]["yellow_red_cards"],
                    "awayteam_yellow_red_cards": match_data[i]["statistics"]["totals"]["competitors"][1]["statistics"]["yellow_red_cards"],
                    "hometeam_red_cards": match_data[i]["statistics"]["totals"]["competitors"][0]["statistics"]["red_cards"],
                    "awayteam_red_cards": match_data[i]["statistics"]["totals"]["competitors"][1]["statistics"]["red_cards"]

                }
        all_match_details.append(match_details)
  return all_match_details

match_summary = extract_match_details(match_data=premier_league_detailed_match_stats)

In [15]:
match_summary[:5]

[{'competition_id': '17',
  'match_id': '50849967',
  'matchday': 1,
  'venue_id': '9',
  'referee': 'Jones, Robert',
  'hometeam_id': '35',
  'awayteam_id': '43',
  'hometeam_ball_possession': 56,
  'awayteam_ball_possession': 44,
  'hometeam_shots_on_target': 5,
  'awayteam_shots_on_target': 2,
  'hometeam_shots_off_target': 7,
  'awayteam_shots_off_target': 4,
  'hometeam_shots_blocked': 2,
  'awayteam_shots_blocked': 4,
  'hometeam_shots_saved': 2,
  'awayteam_shots_saved': 4,
  'hometeam_offsides': 3,
  'awayteam_offsides': 1,
  'hometeam_corner_kicks': 7,
  'awayteam_corner_kicks': 8,
  'hometeam_free_kicks': 11,
  'awayteam_free_kicks': 15,
  'hometeam_cards_given': 2,
  'awayteam_cards_given': 3,
  'hometeam_substitutions': 5,
  'awayteam_substitutions': 5,
  'hometeam_injuries': 0,
  'awayteam_injuries': 0,
  'hometeam_yellow_cards': 2,
  'awayteam_yellow_cards': 3,
  'hometeam_yellow_red_cards': 0,
  'awayteam_yellow_red_cards': 0,
  'hometeam_red_cards': 0,
  'awayteam_red_c

In [16]:
len(match_summary)

380

In [17]:
premier_league_match_summary = pd.DataFrame(match_summary)
premier_league_match_summary.to_csv('/content/drive/My Drive/complete_prem_match_summary.csv', index=False)

premier_league_match_summary.head()

,competition_id,match_id,matchday,venue_id,referee,hometeam_id,awayteam_id,hometeam_ball_possession,awayteam_ball_possession,hometeam_shots_on_target,...,hometeam_substitutions,awayteam_substitutions,hometeam_injuries,awayteam_injuries,hometeam_yellow_cards,awayteam_yellow_cards,hometeam_yellow_red_cards,awayteam_yellow_red_cards,hometeam_red_cards,awayteam_red_cards
0,17,50849967,1,9,"Jones, Robert",35,43,56,44,5,...,5,5,0,0,2,3,0,0,0,0
1,17,50849969,1,2188,"Robinson, Tim",32,44,39,61,2,...,5,4,3,1,3,1,0,0,0,0
2,17,50849971,1,624,"Gillett, Jarred",42,3,53,47,6,...,3,5,0,0,2,2,0,0,0,0
3,17,50849973,1,12,"Hooper, Simon",48,30,37,63,1,...,3,5,0,3,1,1,0,0,1,0
4,17,50849975,1,765,"Pawson, Craig",39,45,22,78,1,...,3,5,1,1,2,4,0,0,1,0
